# Project 1: Simple Web Application CI/CD

This project implements a complete CI/CD pipeline for a single web application, demonstrating foundational concepts from repository structure through production deployment. We will build a Node.js/Express application with automated testing, containerization, security scanning, and progressive deployment to Kubernetes. This end-to-end implementation serves as the baseline pattern that subsequent projects will extend.

## P1.1 Project Overview

**Application**: "Hello-World-API" - A RESTful service providing health checks and greeting endpoints
**Stack**: Node.js 20, Express, PostgreSQL (for persistence demonstration)
**Pipeline**: GitHub Actions → Docker → Amazon ECR → Amazon EKS
**Deployment Strategy**: Blue/Green with automated rollback

**Learning Objectives**:
- Implementing the Twelve-Factor App methodology
- Container image optimization and security scanning
- GitOps-based deployment with ArgoCD
- Automated testing at multiple levels (unit, integration, smoke)
- Secret management with External Secrets Operator
- Observability integration (metrics, logging, tracing)

**Prerequisites**:
- GitHub account with Actions enabled
- AWS account with EKS cluster (v1.27+)
- kubectl configured for cluster access
- Docker Desktop installed locally

## P1.2 Application Structure

The application follows clean architecture principles with clear separation of concerns.

```
hello-world-api/
├── src/
│   ├── config/
│   │   ├── database.js          # Database configuration
│   │   └── index.js             # Environment-based config
│   ├── routes/
│   │   ├── health.js            # Health check endpoints
│   │   └── greeting.js          # Business logic
│   ├── middleware/
│   │   ├── errorHandler.js      # Global error handling
│   │   ├── requestLogger.js     # Structured logging
│   │   └── metrics.js           # Prometheus metrics
│   ├── app.js                   # Express app configuration
│   └── server.js                # Application entry point
├── tests/
│   ├── unit/
│   │   ├── greeting.test.js
│   │   └── health.test.js
│   ├── integration/
│   │   └── api.test.js
│   └── setup.js                 # Test configuration
├── k8s/
│   ├── base/
│   │   ├── deployment.yaml
│   │   ├── service.yaml
│   │   ├── ingress.yaml
│   │   └── kustomization.yaml
│   ├── overlays/
│   │   ├── staging/
│   │   │   ├── patch-deployment.yaml
│   │   │   ├── configmap.yaml
│   │   │   └── kustomization.yaml
│   │   └── production/
│   │       ├── patch-deployment.yaml
│   │       ├── hpa.yaml
│   │       └── kustomization.yaml
│   └── argocd/
│       ├── application.yaml
│       └── app-of-apps.yaml
├── .github/
│   └── workflows/
│       ├── ci.yml               # Continuous Integration
│       ├── cd-staging.yml         # Staging deployment
│       └── cd-production.yml      # Production deployment
├── Dockerfile
├── .dockerignore
├── package.json
└── README.md
```

**Source Code Implementation**:

```javascript
// src/server.js
const app = require('./app');
const config = require('./config');
const logger = require('./config/logger');

const PORT = config.port;

const server = app.listen(PORT, () => {
  logger.info(`Server listening on port ${PORT}`);
  logger.info(`Environment: ${config.env}`);
});

// Graceful shutdown
process.on('SIGTERM', () => {
  logger.info('SIGTERM received, shutting down gracefully');
  server.close(() => {
    logger.info('Process terminated');
    process.exit(0);
  });
});

// src/app.js
const express = require('express');
const helmet = require('helmet');
const cors = require('cors');
const compression = require('compression');

const healthRoutes = require('./routes/health');
const greetingRoutes = require('./routes/greeting');
const errorHandler = require('./middleware/errorHandler');
const requestLogger = require('./middleware/requestLogger');
const metrics = require('./middleware/metrics');

const app = express();

// Security middleware
app.use(helmet());
app.use(cors({ origin: config.allowedOrigins }));
app.use(compression());

// Observability
app.use(requestLogger);
app.use(metrics.middleware);

// Body parsing
app.use(express.json());
app.use(express.urlencoded({ extended: true }));

// Routes
app.use('/health', healthRoutes);
app.use('/api/v1/greeting', greetingRoutes);

// Metrics endpoint for Prometheus
app.get('/metrics', metrics.endpoint);

// Global error handling
app.use(errorHandler);

module.exports = app;
```

```javascript
// src/config/index.js
// Twelve-Factor: Config in environment
const config = {
  env: process.env.NODE_ENV || 'development',
  port: parseInt(process.env.PORT, 10) || 3000,
  db: {
    host: process.env.DB_HOST,
    port: parseInt(process.env.DB_PORT, 10) || 5432,
    database: process.env.DB_NAME,
    user: process.env.DB_USER,
    password: process.env.DB_PASSWORD,
    ssl: process.env.DB_SSL === 'true',
    poolSize: parseInt(process.env.DB_POOL_SIZE, 10) || 10
  },
  logLevel: process.env.LOG_LEVEL || 'info',
  allowedOrigins: process.env.ALLOWED_ORIGINS?.split(',') || ['http://localhost:3000']
};

// Validation
if (!config.db.host && config.env === 'production') {
  throw new Error('DB_HOST is required in production');
}

module.exports = config;
```

## P1.3 Dockerizing the Application

Multi-stage build optimizing for size and security.

**Dockerfile**:
```dockerfile
# syntax=docker/dockerfile:1
FROM node:20-alpine AS builder

WORKDIR /app

# Install dependencies first (caching)
COPY package*.json ./
RUN npm ci --only=production && npm cache clean --force

# Production stage
FROM node:20-alpine AS production

# Security: Run as non-root
RUN addgroup -g 1001 -S nodejs && \
    adduser -S nodejs -u 1001

WORKDIR /app

# Copy dependencies from builder
COPY --from=builder --chown=nodejs:nodejs /app/node_modules ./node_modules

# Copy application code
COPY --chown=nodejs:nodejs . .

# Switch to non-root user
USER nodejs

# Expose port
EXPOSE 3000

# Health check
HEALTHCHECK --interval=30s --timeout=3s --start-period=5s --retries=3 \
  CMD node -e "require('http').get('http://localhost:3000/health/live', (r) => {process.exit(r.statusCode === 200 ? 0 : 1)})"

CMD ["node", "src/server.js"]
```

**.dockerignore**:
```
node_modules
npm-debug.log
Dockerfile
.dockerignore
.git
.gitignore
README.md
.env
.nyc_output
coverage
tests
*.test.js
.github
k8s
docs
```

**Image Verification**:
```bash
# Build and test locally
docker build -t hello-world-api:latest .

# Check image size (target: <200MB)
docker images hello-world-api:latest

# Run locally with env vars
docker run -p 3000:3000 \
  -e NODE_ENV=production \
  -e PORT=3000 \
  hello-world-api:latest

# Test health endpoint
curl http://localhost:3000/health/live
```

## P1.4 Creating Kubernetes Manifests

Base manifests with Kustomize overlays for environment-specific configuration.

**Base Deployment** (`k8s/base/deployment.yaml`):
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-api
  labels:
    app: hello-world-api
    version: v1
spec:
  replicas: 2
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 25%
      maxUnavailable: 0
  selector:
    matchLabels:
      app: hello-world-api
  template:
    metadata:
      labels:
        app: hello-world-api
        version: v1
      annotations:
        prometheus.io/scrape: "true"
        prometheus.io/port: "3000"
        prometheus.io/path: "/metrics"
    spec:
      serviceAccountName: hello-world-api
      securityContext:
        runAsNonRoot: true
        runAsUser: 1001
        fsGroup: 1001
      containers:
      - name: app
        image: hello-world-api
        imagePullPolicy: Always
        ports:
        - containerPort: 3000
          name: http
          protocol: TCP
        env:
        - name: NODE_ENV
          value: "production"
        - name: PORT
          value: "3000"
        - name: DB_HOST
          valueFrom:
            configMapKeyRef:
              name: db-config
              key: host
        - name: DB_PASSWORD
          valueFrom:
            secretKeyRef:
              name: db-credentials
              key: password
        resources:
          requests:
            memory: "128Mi"
            cpu: "100m"
          limits:
            memory: "512Mi"
            cpu: "500m"
        securityContext:
          allowPrivilegeEscalation: false
          readOnlyRootFilesystem: true
          capabilities:
            drop:
            - ALL
        livenessProbe:
          httpGet:
            path: /health/live
            port: 3000
          initialDelaySeconds: 10
          periodSeconds: 10
          timeoutSeconds: 5
          failureThreshold: 3
        readinessProbe:
          httpGet:
            path: /health/ready
            port: 3000
          initialDelaySeconds: 5
          periodSeconds: 5
          timeoutSeconds: 3
          failureThreshold: 3
        volumeMounts:
        - name: tmp
          mountPath: /tmp
      volumes:
      - name: tmp
        emptyDir: {}
```

**Base Service** (`k8s/base/service.yaml`):
```yaml
apiVersion: v1
kind: Service
metadata:
  name: hello-world-api
  labels:
    app: hello-world-api
spec:
  type: ClusterIP
  ports:
  - port: 80
    targetPort: 3000
    protocol: TCP
    name: http
  selector:
    app: hello-world-api
```

**Production Overlay** (`k8s/overlays/production/patch-deployment.yaml`):
```yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-world-api
spec:
  replicas: 4
  template:
    spec:
      containers:
      - name: app
        resources:
          requests:
            memory: "256Mi"
            cpu: "250m"
          limits:
            memory: "1Gi"
            cpu: "1000m"
        env:
        - name: LOG_LEVEL
          value: "warn"
```

**Production HPA** (`k8s/overlays/production/hpa.yaml`):
```yaml
apiVersion: autoscaling/v2
kind: HorizontalPodAutoscaler
metadata:
  name: hello-world-api
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: hello-world-api
  minReplicas: 4
  maxReplicas: 20
  metrics:
  - type: Resource
    resource:
      name: cpu
      target:
        type: Utilization
        averageUtilization: 70
  - type: Resource
    resource:
      name: memory
      target:
        type: Utilization
        averageUtilization: 80
  behavior:
    scaleUp:
      stabilizationWindowSeconds: 60
      policies:
      - type: Percent
        value: 100
        periodSeconds: 15
    scaleDown:
      stabilizationWindowSeconds: 300
      policies:
      - type: Percent
        value: 10
        periodSeconds: 60
```

**External Secret** (`k8s/overlays/production/externalsecret.yaml`):
```yaml
apiVersion: external-secrets.io/v1beta1
kind: ExternalSecret
metadata:
  name: db-credentials
spec:
  refreshInterval: 1h
  secretStoreRef:
    kind: ClusterSecretStore
    name: aws-secrets-manager
  target:
    name: db-credentials
    creationPolicy: Owner
  data:
  - secretKey: password
    remoteRef:
      key: prod/hello-world-api/db-password
```

## P1.5 Building the CI Pipeline

Continuous Integration workflow with comprehensive testing and security scanning.

**`.github/workflows/ci.yml`**:
```yaml
name: Continuous Integration

on:
  push:
    branches: [main]
    paths:
      - 'src/**'
      - 'tests/**'
      - 'package*.json'
      - 'Dockerfile'
      - '.github/workflows/ci.yml'
  pull_request:
    branches: [main]
    paths:
      - 'src/**'
      - 'tests/**'
      - 'package*.json'
      - 'Dockerfile'

env:
  NODE_VERSION: '20'
  REGISTRY: ${{ secrets.AWS_ACCOUNT_ID }}.dkr.ecr.${{ secrets.AWS_REGION }}.amazonaws.com
  IMAGE_NAME: hello-world-api

jobs:
  # Job 1: Fast feedback - Lint and unit tests
  lint-and-unit-test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Node.js
        uses: actions/setup-node@v4
        with:
          node-version: ${{ env.NODE_VERSION }}
          cache: 'npm'
      
      - name: Install dependencies
        run: npm ci
      
      - name: Run linter
        run: npm run lint
      
      - name: Run unit tests
        run: npm run test:unit -- --coverage
      
      - name: Upload coverage
        uses: codecov/codecov-action@v3
        with:
          files: ./coverage/lcov.info
          fail_ci_if_error: true

  # Job 2: Security scanning (parallel)
  security-scan:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Run npm audit
        run: npm audit --audit-level=high
      
      - name: Run Trivy vulnerability scanner
        uses: aquasecurity/trivy-action@master
        with:
          scan-type: 'fs'
          scan-ref: '.'
          severity: 'CRITICAL,HIGH'
          exit-code: '1'
      
      - name: Check for secrets
        uses: trufflesecurity/trufflehog@main
        with:
          path: ./
          base: main
          head: HEAD
          extra_args: --debug --only-verified

  # Job 3: Integration tests with database
  integration-test:
    runs-on: ubuntu-latest
    services:
      postgres:
        image: postgres:15-alpine
        env:
          POSTGRES_PASSWORD: postgres
          POSTGRES_DB: testdb
        options: >-
          --health-cmd pg_isready
          --health-interval 10s
          --health-timeout 5s
          --health-retries 5
        ports:
          - 5432:5432
    steps:
      - uses: actions/checkout@v4
      
      - name: Setup Node.js
        uses: actions/setup-node@v4
        with:
          node-version: ${{ env.NODE_VERSION }}
          cache: 'npm'
      
      - name: Install dependencies
        run: npm ci
      
      - name: Run integration tests
        run: npm run test:integration
        env:
          DB_HOST: localhost
          DB_PORT: 5432
          DB_NAME: testdb
          DB_USER: postgres
          DB_PASSWORD: postgres
          DB_SSL: false

  # Job 4: Build and push image
  build-and-push:
    needs: [lint-and-unit-test, security-scan]
    runs-on: ubuntu-latest
    permissions:
      id-token: write  # For OIDC
      contents: read
    steps:
      - uses: actions/checkout@v4
      
      - name: Configure AWS credentials
        uses: aws-actions/configure-aws-credentials@v4
        with:
          role-to-assume: arn:aws:iam::${{ secrets.AWS_ACCOUNT_ID }}:role/github-actions-ecr
          aws-region: ${{ secrets.AWS_REGION }}
      
      - name: Login to Amazon ECR
        id: login-ecr
        uses: aws-actions/amazon-ecr-login@v2
      
      - name: Set up Docker Buildx
        uses: docker/setup-buildx-action@v3
      
      - name: Build and export to Docker
        uses: docker/build-push-action@v5
        with:
          context: .
          load: true
          tags: ${{ env.IMAGE_NAME }}:${{ github.sha }}
      
      - name: Scan Docker image
        uses: aquasecurity/trivy-action@master
        with:
          image-ref: ${{ env.IMAGE_NAME }}:${{ github.sha }}
          format: 'sarif'
          output: 'trivy-results.sarif'
          severity: 'CRITICAL,HIGH'
          exit-code: '1'
      
      - name: Upload scan results
        uses: github/codeql-action/upload-sarif@v2
        if: always()
        with:
          sarif_file: 'trivy-results.sarif'
      
      - name: Generate SBOM
        uses: anchore/sbom-action@v0
        with:
          image: ${{ env.IMAGE_NAME }}:${{ github.sha }}
          format: spdx-json
          output-file: sbom.spdx.json
      
      - name: Push image to ECR
        uses: docker/build-push-action@v5
        with:
          context: .
          push: true
          tags: |
            ${{ steps.login-ecr.outputs.registry }}/${{ env.IMAGE_NAME }}:${{ github.sha }}
            ${{ steps.login-ecr.outputs.registry }}/${{ env.IMAGE_NAME }}:latest
          cache-from: type=gha
          cache-to: type=gha,mode=max
      
      - name: Sign image with Cosign
        uses: sigstore/cosign-installer@v3
      - run: |
          cosign sign \
            --yes \
            --key env://COSIGN_PRIVATE_KEY \
            ${{ steps.login-ecr.outputs.registry }}/${{ env.IMAGE_NAME }}:${{ github.sha }}
        env:
          COSIGN_PRIVATE_KEY: ${{ secrets.COSIGN_PRIVATE_KEY }}
          COSIGN_PASSWORD: ${{ secrets.COSIGN_PASSWORD }}
      
      - name: Upload artifacts
        uses: actions/upload-artifact@v4
        with:
          name: build-artifacts
          path: |
            sbom.spdx.json
            trivy-results.sarif
```

## P1.6 Building the CD Pipeline

GitOps-based deployment with ArgoCD and automated smoke tests.

**ArgoCD Application** (`k8s/argocd/application.yaml`):
```yaml
apiVersion: argoproj.io/v1alpha1
kind: Application
metadata:
  name: hello-world-api
  namespace: argocd
  finalizers:
    - resources-finalizer.argocd.argoproj.io
spec:
  project: production
  source:
    repoURL: https://github.com/myorg/hello-world-api.git
    targetRevision: HEAD
    path: k8s/overlays/production
  destination:
    server: https://kubernetes.default.svc
    namespace: production
  syncPolicy:
    automated:
      prune: true
      selfHeal: true
      allowEmpty: false
    syncOptions:
      - CreateNamespace=true
      - PrunePropagationPolicy=foreground
      - PruneLast=true
    retry:
      limit: 5
      backoff:
        duration: 5s
        factor: 2
        maxDuration: 3m
```

**Staging Deployment Workflow** (`.github/workflows/cd-staging.yml`):
```yaml
name: Deploy to Staging

on:
  push:
    branches: [main]
    paths:
      - 'src/**'
      - 'k8s/**'
  workflow_dispatch:

jobs:
  deploy-staging:
    runs-on: ubuntu-latest
    environment: staging
    steps:
      - uses: actions/checkout@v4
      
      - name: Update image tag
        run: |
          cd k8s/overlays/staging
          kustomize edit set image hello-world-api=${{ secrets.ECR_REGISTRY }}/hello-world-api:${{ github.sha }}
      
      - name: Commit changes
        run: |
          git config --local user.email "action@github.com"
          git config --local user.name "GitHub Action"
          git add k8s/overlays/staging/kustomization.yaml
          git commit -m "Update staging image to ${{ github.sha }}"
          git push

  smoke-test-staging:
    needs: deploy-staging
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      
      - name: Configure kubectl
        run: |
          aws eks update-kubeconfig --name staging-cluster
      
      - name: Wait for rollout
        run: |
          kubectl rollout status deployment/hello-world-api -n staging --timeout=300s
      
      - name: Run smoke tests
        run: |
          ENDPOINT=$(kubectl get ingress hello-world-api -n staging -o jsonpath='{.status.loadBalancer.ingress[0].hostname}')
          
          # Health check
          curl -f http://$ENDPOINT/health/live || exit 1
          
          # API test
          curl -f http://$ENDPOINT/api/v1/greeting?name=Test || exit 1
          
          # Metrics check
          curl -f http://$ENDPOINT/metrics || exit 1
      
      - name: Notify on failure
        if: failure()
        uses: slackapi/slack-github-action@v1
        with:
          payload: |
            {
              "text": "Staging deployment failed for hello-world-api"
            }
        env:
          SLACK_WEBHOOK_URL: ${{ secrets.SLACK_WEBHOOK }}
```

**Production Deployment Workflow** (`.github/workflows/cd-production.yml`):
```yaml
name: Deploy to Production

on:
  workflow_dispatch:
    inputs:
      image_tag:
        description: 'Image tag to deploy (default: latest commit SHA)'
        required: false
        default: ''

jobs:
  deploy-production:
    runs-on: ubuntu-latest
    environment: production  # Requires manual approval
    steps:
      - uses: actions/checkout@v4
      
      - name: Configure AWS credentials
        uses: aws-actions/configure-aws-credentials@v4
        with:
          role-to-assume: arn:aws:iam::${{ secrets.AWS_ACCOUNT_ID }}:role/github-actions-eks
          aws-region: ${{ secrets.AWS_REGION }}
      
      - name: Verify image signature
        uses: sigstore/cosign-installer@v3
      - run: |
          IMAGE_TAG=${{ github.event.inputs.image_tag || github.sha }}
          cosign verify \
            --key env://COSIGN_PUBLIC_KEY \
            ${{ secrets.ECR_REGISTRY }}/hello-world-api:$IMAGE_TAG
        env:
          COSIGN_PUBLIC_KEY: ${{ secrets.COSIGN_PUBLIC_KEY }}
      
      - name: Update image tag in GitOps repo
        run: |
          cd k8s/overlays/production
          IMAGE_TAG=${{ github.event.inputs.image_tag || github.sha }}
          kustomize edit set image hello-world-api=${{ secrets.ECR_REGISTRY }}/hello-world-api:$IMAGE_TAG
      
      - name: Create Pull Request
        uses: peter-evans/create-pull-request@v5
        with:
          token: ${{ secrets.GITHUB_TOKEN }}
          commit-message: "Deploy hello-world-api to production"
          title: "Production deployment: hello-world-api"
          body: |
            ## Deployment Details
            - **Image Tag**: ${{ github.event.inputs.image_tag || github.sha }}
            - **Triggered by**: ${{ github.actor }}
            - **Pipeline**: ${{ github.run_id }}
            
            ## Checklist
            - [ ] Staging tests passed
            - [ ] Change ticket approved
            - [ ] Database migrations reviewed (if applicable)
            
            ArgoCD will automatically sync after merge.
          branch: deploy/production-${{ github.run_id }}
          delete-branch: true

  verify-production:
    needs: deploy-production
    runs-on: ubuntu-latest
    steps:
      - name: Configure kubectl
        run: |
          aws eks update-kubeconfig --name production-cluster
      
      - name: Verify deployment
        run: |
          kubectl rollout status deployment/hello-world-api -n production --timeout=600s
          
          # Verify ArgoCD sync
          argocd app wait hello-world-api --health --timeout 300
      
      - name: Run production smoke tests
        run: |
          ENDPOINT=https://api.company.com
          
          # Comprehensive health check
          curl -f $ENDPOINT/health/live
          curl -f $ENDPOINT/health/ready
          
          # Business logic
          RESPONSE=$(curl -s $ENDPOINT/api/v1/greeting?name=Production)
          echo "Response: $RESPONSE"
          
          # Verify in Datadog/NewRelic
          # Check error rates < 0.1%
      
      - name: Rollback on failure
        if: failure()
        run: |
          echo "Deployment failed, initiating rollback..."
          # ArgoCD will auto-sync to previous Git state
          # Or manual: kubectl rollout undo deployment/hello-world-api -n production
```

## P1.7 Deployment and Testing

Step-by-step execution and verification.

**Local Development Setup**:
```bash
# Clone and setup
git clone https://github.com/myorg/hello-world-api.git
cd hello-world-api
npm install

# Run with Docker Compose
docker-compose up -d  # Starts app + PostgreSQL

# Run tests
npm test
npm run test:integration

# Build and run container locally
docker build -t hello-world-api:local .
docker run -p 3000:3000 --env-file .env hello-world-api:local
```

**Staging Deployment**:
1. Push to `main` branch triggers CI
2. CI completes successfully, updates GitOps repo
3. ArgoCD detects change, syncs to staging cluster
4. Smoke tests verify deployment
5. Slack notification confirms success

**Production Deployment**:
1. Manual workflow dispatch with image tag
2. Requires environment approval (manual gate)
3. Verifies image signature with Cosign
4. Creates PR to update production overlay
5. Merge triggers ArgoCD sync
6. Automated verification tests
7. Automatic rollback if health checks fail

**Verification Commands**:
```bash
# Check deployment status
kubectl get pods -n production -l app=hello-world-api
kubectl get svc -n production hello-world-api
kubectl get ingress -n production hello-world-api

# View logs
kubectl logs -n production -l app=hello-world-api --tail=100

# Port forward for local testing
kubectl port-forward -n production svc/hello-world-api 8080:80
curl http://localhost:8080/health/live

# Check ArgoCD sync
argocd app get hello-world-api
```

## P1.8 Project Summary

This project implemented a complete CI/CD pipeline demonstrating:

**Technical Achievements**:
- **12-Factor compliance**: Config externalization, stateless processes, port binding
- **Security**: Non-root containers, vulnerability scanning, image signing, secret management
- **Observability**: Prometheus metrics, structured logging, health checks
- **GitOps**: Declarative infrastructure, automated sync, drift detection

**Pipeline Characteristics**:
- **Fast feedback**: Lint and unit tests complete in <2 minutes
- **Parallel execution**: Security scans run concurrently with integration tests
- **Quality gates**: Coverage thresholds, vulnerability blocking, signature verification
- **Progressive deployment**: Staging → Production with manual approval

**Operational Excellence**:
- **Self-healing**: Kubernetes probes, ArgoCD auto-sync
- **Scalability**: HPA for production workloads
- **Reliability**: Multi-stage Docker builds, resource limits, graceful shutdown
- **Maintainability**: Kustomize overlays, modular manifests, clear documentation

**Next Steps**:
In Project 2, we extend these patterns to a microservices architecture, addressing service-to-service communication, contract testing, and multi-service deployment coordination.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../1. foundations/1. introduction_to_cicd.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../1. foundations/2. essential_concepts_and_terminology.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
